[Improved H&M Pure Pytorch Baseline](https://www.kaggle.com/code/guoyonfan/improved-h-m-pure-pytorch-baseline/notebook)

# Data Load


In [ ]:
import gc
import numpy as np
import pandas as pd

is_debug = False
df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv",
                 dtype={"article_id": str})
print(df.shape)
df.head()

# t_dat 연산 할 수 있게 날짜형식으로 변경

In [ ]:
df["t_dat"] = pd.to_datetime(df["t_dat"])
df["t_dat"].max()

# 구매 날짜가 가장 최근인 고객 ID를 그룹화 시킴
# 2019년 9월 1일 이후로 필터링

In [ ]:
active_articles = df.groupby("article_id")["t_dat"].max().reset_index()
active_articles = active_articles[active_articles["t_dat"] >= "2019-09-01"].reset_index()
active_articles.shape

## 고객 ID가 존재하는지 필터링

In [ ]:
df = df[df["article_id"].isin(active_articles["article_id"])].reset_index(drop=True)
df.shape

NameError: name 'df' is not defined

# 가장 최근 날짜로 부터 각 날짜의 일수를 구함 (.dt.days를 이용하여 일수를 주로 변환)

In [ ]:
df["week"] = (df["t_dat"].max() - df["t_dat"]).dt.days // 7
df["week"].value_counts()

# Label Encoder를 이용하여 고유한 고객 id를 추출 후 정수로 변환하여 ML 데이터 변환
# LabelEncoder는 문자열(범주형 데이터)을 정수로 변환하는 데 사용

In [ ]:
from sklearn.preprocessing import LabelEncoder


article_ids = np.concatenate([["placeholder"], np.unique(df["article_id"].values)])

le_article = LabelEncoder()
le_article.fit(article_ids)
df["article_id"] = le_article.transform(df["article_id"])

# 과거 5주까지 데이터를 바탕으로 고객, 물품 간의 관계를 나타내는 훈련 데이터셋 생성

In [ ]:
WEEK_HIST_MAX = 5

def create_dataset(df, week):
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index() # customer_id로 그룹화 후 물품 id와 주를 리스트 형식으로 묶음
    hist_df.rename(columns={"week": 'week_history'}, inplace=True) # week 이름을 week_history로 변경

    target_df = df[df["week"] == week] # 정확한 주를 선택하여 데이터 셋 제작
    target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index() # customer_id로 그룹화 후 물품 id와 리스트 형식으로 묶음
    target_df.rename(columns={"article_id": "target"}, inplace=True) # 물품 id를 target으로 변경
    target_df["week"] = week # 타겟 데이터에 선택한 week 추가

    return target_df.merge(hist_df, on="customer_id", how="left") # 과거 주 데이터와 target 데이터 병합

# val_weeks = [0]
train_weeks = [i for i in range(WEEK_HIST_MAX)]

# val_df = pd.concat([create_dataset(df, w) for w in val_weeks]).reset_index(drop=True)
train_df = pd.concat([create_dataset(df, w) for w in train_weeks]).reset_index(drop=True)
if is_debug:
    train_df = train_df.sample(n = 10000)

del df
print(gc.collect())

train_articles = train_df['customer_id'].unique().tolist() # 고유한 customer_id를 리스트로 만들고 train_articles에 저장
train_df.shape

# HM Model를 이용하여 모델 생성

## HM Model이란?
  - PyTorch를 사용하여 만든 딥러닝 모델
  - input : 고객 구매이력과 시간 정보(week)
  - output : 각 물품에 추천 점수를 생성함(CNN 모델을 통해 물품에 대한 최종 추천 점수를 출력)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

class HMDataset(Dataset):
    def __init__(self, df, seq_len, is_test=False):
        self.df = df.reset_index(drop=True)
        self.seq_len = seq_len
        self.is_test = is_test

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]

        if self.is_test:
            target = torch.zeros(2).float()
        else:
            target = torch.zeros(len(article_ids)).float()
            for t in row.target:
                target[t] = 1.0

        article_hist = torch.zeros(self.seq_len).long()
        week_hist = torch.ones(self.seq_len).float()


        if isinstance(row.article_id, list):
            if len(row.article_id) >= self.seq_len:
                article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
                week_hist = (torch.LongTensor(row.week_history[-self.seq_len:]) - row.week)/WEEK_HIST_MAX/2
            else:
                article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                week_hist[-len(row.article_id):] = (torch.LongTensor(row.week_history) - row.week)/WEEK_HIST_MAX/2

        return article_hist, week_hist, target
